In [1]:
import os
import json
import time
import argparse

import torch

from typing import Dict, List, Tuple, Set, Optional

from prefetch_generator import BackgroundGenerator
from tqdm import tqdm

from torch.optim import Adam, SGD
from pytorch_transformers import AdamW, WarmupLinearSchedule

from lib.preprocessings import ScienceIE_selection_preprocessing
from lib.dataloaders import Selection_Dataset, Selection_loader, Unlabelled_Selection_Dataset, Unlabelled_Selection_loader
from lib.metrics import F1_triplet, F1_ner
from lib.models import MultiHeadSelection
from lib.config import Hyper

In [2]:
MODEL_DIRECTORY = 'saved_models/scienceie_scibert_crf_grp_mean_lr_same'
MODEL_NAME = 'best_model.pt'

hyper = Hyper(os.path.join('experiments', 'scienceie_selection_re.json'))
hyper.global_relation_prediction = 1
hyper.model_dir = MODEL_DIRECTORY
hyper.gpu = 1

In [3]:
model = MultiHeadSelection(hyper).cuda('cuda:1')

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
model.load_state_dict(torch.load(os.path.join(MODEL_DIRECTORY, MODEL_NAME)))

<All keys matched successfully>

In [5]:
dev_set = Selection_Dataset(hyper, hyper.dev)
loader = Selection_loader(dev_set, batch_size=hyper.eval_batch, pin_memory=False)
unlabelled_set = Unlabelled_Selection_Dataset(hyper, "unlabelled_original_scibert.txt", "unlabelled_augmented_scibert.txt")
unlabelled_loader = Unlabelled_Selection_loader(unlabelled_set, batch_size=hyper.eval_batch, pin_memory=False)
model.eval()

MultiHeadSelection(
  (bert_embeddings): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31116, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [6]:
def masked_BCEloss(selection_mask, selection_logits, selection_gold):
    # convert true/false to 1/0
    selection_loss = F.binary_cross_entropy_with_logits(selection_logits,
                                                            selection_gold,
                                                            reduction='none')
    selection_loss = selection_loss.masked_select(selection_mask).sum()
    selection_loss /= selection_mask.sum() # here is different
    return selection_loss

for i, sample in enumerate(BackgroundGenerator(unlabelled_loader)):
    # pseudo label from weakly augmented
    sample.text = sample.dataset1
    sample.tokens = sample.tokens1
    with torch.no_grad():
        output = model(sample, is_train=False, is_inference=True)
    ssl_bio_tag = output['predicted_tag']
    ssl_bio_mask = output['bio_mask']
    ssl_selection_tag = output['selection_gold_logits']
    ssl_selection_mask = output['selection_mask']
    ssl_grp_tag = output['grp_gold_logits']
    
    # logits from strongly augmented
    sample.text = sample.dataset2
    sample.tokens = sample.tokens2
    output = model(sample, is_train=False, is_inference=True)
    ssl_emission = output['emission_tag']
    ssl_selection_logits = output['selection_logits']
    ssl_grp_logits = output['grp_logits']
    
    # compute loss
    ssl_crf_loss = model.tagger(ssl_emission, ssl_bio_tag, mask=ssl_bio_mask, reduction='mean')
    ssl_selection_loss = masked_BCEloss(ssl_selection_mask, ssl_selection_logits, ssl_selection_tag)
    ssl_grp_loss = nn.CrossEntropyLoss()(ssl_grp_logits.view(-1, 2), ssl_grp_tag.view(-1))
    ssl_loss = ssl_crf_loss + ssl_selection_loss + ssl_grp_loss

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


NameError: name 'Error' is not defined

In [13]:
print(output.keys())
print("-----")
print(output['predicted_tag'])

dict_keys(['predicted_tag', 'emission_tag', 'bio_mask', 'decoded_tag', 'selection_triplets', 'selection_logits', 'selection_gold_logits', 'selection_mask', 'grp_logits', 'grp_gold_logits', 'crf_loss', 'selection_loss', 'grp_loss', 'loss', 'description'])
-----
[[2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 2, 2, 0, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [4]:
hyper = Hyper(os.path.join('experiments', 'scienceie_selection_re.json'))
unlabelled_set = Unlabelled_Selection_Dataset(hyper, "unlabelled_original_scibert2.txt", "unlabelled_augmented_scibert2.txt")
unlabelled_loader = Unlabelled_Selection_loader(unlabelled_set, batch_size=hyper.ssl_batch, pin_memory=False)

In [5]:
for sample in unlabelled_loader:
    print(sample.dataset1)
    break

(['a', 'bond', 'failure', 'model', 'thought', 'of', 'as', 'a', 'micro', '-', 'crack', 'nucleation', ',', 'specifically', 'following', 'a', 'separation', 'between', 'the', 'adjacent', 'cells', 'in', 'the', 'cellular', 'structure', 'along', 'one', 'common', 'face', '.'], ['initially', ',', 'small', 'micro', '-', 'cracks', 'may', 'be', 'dispersed', 'in', 'the', 'model', 'because', 'likely', 'random', 'distribution', 'of', 'pore', 'sizes', 'and', 'the', 'low', 'level', 'of', 'interaction', 'due', 'to', 'force', 'redistribution', '.'])


In [6]:
batches = iter(unlabelled_loader)

In [7]:
idx = 1
while True:
    print(idx)
    idx += 1
    next(batches)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195


StopIteration: 

In [40]:
print(first.dataset1)

(['One', 'addition', ',', 'the', 'detection', 'of', 'solar', 'cell', '’', 's', 'temperature', 'is', 'very', 'useful', 'for', 'the', 'electrical', 'characterisation', 'of', 'tunnel', 'diodes', '.'], ['m', 'et', 'r.'], ['(', 'C', ')', 'Four', 'potential', 'methods', 'for', 'the', 'calculation', 'of', 'the', 'cell', 'temperature', 'in'], ['Treatment', 'dose', 'pv', '(', 'hcpv', ')', 'A.'], ['the', 'methods', 'were', 'categorised', 'based', 'on', ':', '(', '1', ')', 'classification', 'by', 'temperature', ',', '(', '1', ')', 'environmental', 'parameters', 'und', '(', '3', ')', 'atmospheric', 'properties', '.'], ['the', 'first', 'few', 'categories', 'are', 'based', 'on', 'direct', 'measurements', 'of', 'Tb', 'values', 'using', 'indoor', 'versus', 'outdoor', 'experimental', 'setups', 'and', 'presented', 'with', 'highest', 'degree', 'of', 'accuracy'], ['(', 'round', 'mean', 'squares', 'error', '(', 'rmse', ')', '1', '.', '7', 'and', '2', '.', '2', 'k', ')', '.'], ['most', 'of', 'these', 'study